(mmt-tune)=

# Batch training & tuning on Ray Tune

**Batch training** and tuning are common tasks in simple machine learning use-cases such as time series forecasting. They require fitting of simple models on multiple data batches corresponding to locations, products, etc.

**Batch training** is a workload that trains model(s) on subsets of a dataset. This notebook showcases how to conduct batch training using [Ray Tune](https://docs.ray.io/en/latest/tune/index.html).

![Batch training diagram](../../data/examples/images/batch-training.svg)

For the data, we will use the [NYC Taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). This popular tabular dataset contains historical taxi pickups by timestamp and location in NYC.

In the notebook, we will split the data by dropoff location and train a separate regression model for each dropoff location predicting `trip_duration`. Specifically, we will use the `dropoff_location_id` column in the dataset to group the dataset into data batches. Then we will fit a separate model for each batch and evaluate it.

# Contents

In this this tutorial, you will learn about:
 1. [Define how to load and prepare Parquet data](#load_data)
 2. [Define your Ray Tune Search Space and Search Algorithm](#define_search_space)
 3. [Define a Trainable (callable) function](#define_trainable)
 4. [Run batch training on Ray Tune](#run_tune_search)
 5. [Load a model from checkpoint and perform inference](#load_checkpoint)


# Walkthrough

```{tip}
Prerequisite for this notebook: Read the [Key Concepts](https://docs.ray.io/en/latest/tune/key-concepts.html) page for Ray Tune.
```

Let us start by importing a few required libraries, including open-source [Ray](https://github.com/ray-project/ray) itself!

In [1]:
import os

print(f"Number of CPUs in this system: {os.cpu_count()}")
from typing import Tuple, List, Union, Optional, Callable
import time
import pandas as pd
import numpy as np
import pyarrow
import pyarrow.parquet as pq
import pyarrow.dataset as pds

print(f"pyarrow: {pyarrow.__version__}")

Number of CPUs in this system: 16
pyarrow: 10.0.0


In [2]:
import ray

if ray.is_initialized():
    ray.shutdown()
ray.init()

Python version:,3.8.5
Ray version:,2.1.0
Dashboard:,http://console.anyscale-staging.com/api/v2/sessions/ses_z9YHnwAJ9GkmbH4ZEeFfjQsc/services?redirect_to=dashboard


In [3]:
print(ray.cluster_resources())

{'CPU': 32.0, 'memory': 42071657678.0, 'node:172.31.92.227': 1.0, 'object_store_memory': 23720183398.0, 'node:172.31.119.49': 1.0}


In [4]:
# import standard sklearn libraries
import sklearn
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

print(f"sklearn: {sklearn.__version__}")

# import ray libraries
from ray import air, tune
from ray.air import session
from ray.air.checkpoint import Checkpoint

# set global random seed for sklearn models
np.random.seed(415)

sklearn: 1.1.3


In [5]:
# For benchmarking purposes, we can print the times of various operations.
# In order to reduce clutter in the output, this is set to False by default.
PRINT_TIMES = False


def print_time(msg: str):
    if PRINT_TIMES:
        print(msg)


# To speed things up, we’ll only use a small subset of the full dataset consisting of two last months of 2019.
# You can choose to use the full dataset for 2018-2019 by setting the SMOKE_TEST variable to False.
SMOKE_TEST = True

## Define how to load and prepare Parquet data <a class="anchor" id="load_data"></a>

First, we need to load some data.  Since the NYC Taxi dataset is fairly large, we will filter files first into a PyArrow dataset. And then in the next cell after, we will filter the data on read into a PyArrow table and convert that to a pandas dataframe.

```{tip}
Use PyArrow dataset and table for reading or writing large parquet files, since its native multithreaded C++ adpater is faster than pandas read_parquet, even using engine=pyarrow.
```

In [6]:
# Define some global variables.
target = "trip_duration"
s3_partitions = pds.dataset(
    "s3://anonymous@air-example-data/ursa-labs-taxi-data/by_year/",
    partitioning=["year", "month"],
)
s3_files = [f"s3://anonymous@{file}" for file in s3_partitions.files]

# Obtain all location IDs
all_location_ids = (
    pq.read_table(s3_files[0], columns=["dropoff_location_id"])["dropoff_location_id"]
    .unique()
    .to_pylist()
)
# drop [264, 265]
all_location_ids.remove(264)
all_location_ids.remove(265)

# Use smoke testing or not.
starting_idx = -1 if SMOKE_TEST else 0
# note: use location 199 to test error-handling
sample_locations = [90, 145, 152, 204] if SMOKE_TEST else all_location_ids

# Display what data will be used.
s3_files = s3_files[starting_idx:]
print(f"NYC Taxi using {len(s3_files)} file(s)!")
print(f"s3_files: {s3_files}")
print(f"Locations: {sample_locations}")

# Display what data will be used.
s3_files = s3_files[starting_idx:]
print(f"NYC Taxi using {len(s3_files)} file(s)!")
print(f"s3_files: {s3_files}")
print(f"Locations: {sample_locations}")

NYC Taxi using 18 file(s)!
s3_files: ['s3://air-example-data/ursa-labs-taxi-data/by_year/2018/01/data.parquet/4d6bc4368704460d90c92c22e05a2220_000000.parquet', 's3://air-example-data/ursa-labs-taxi-data/by_year/2018/02/data.parquet/e817946252d1409b93964685130e76cb_000000.parquet', 's3://air-example-data/ursa-labs-taxi-data/by_year/2018/03/data.parquet/0b7e5121a4904c64be5e91ceec0eee2f_000000.parquet', 's3://air-example-data/ursa-labs-taxi-data/by_year/2018/04/data.parquet/f40c2c2806e548bfac8336de9c19a423_000000.parquet', 's3://air-example-data/ursa-labs-taxi-data/by_year/2018/05/data.parquet/a5de27164fda47988dec2576685656ae_000000.parquet', 's3://air-example-data/ursa-labs-taxi-data/by_year/2018/06/data.parquet/df104576ffed4e308b72941df90f7790_000000.parquet', 's3://air-example-data/ursa-labs-taxi-data/by_year/2018/07/data.parquet/ccdef45e50de4678b7e589155f372a3d_000000.parquet', 's3://air-example-data/ursa-labs-taxi-data/by_year/2018/08/data.parquet/9d605bf8abf84655997d491bc5a10a4c_000

In [7]:
# Function to read a pyarrow.Table object using pyarrow parquet
def read_data(file: str, sample_id: np.int32) -> pd.DataFrame:

    df = pq.read_table(
        file,
        filters=[
            ("passenger_count", ">", 0),
            ("trip_distance", ">", 0),
            ("fare_amount", ">", 0),
            ("pickup_location_id", "not in", [264, 265]),
            ("dropoff_location_id", "not in", [264, 265]),
            ("dropoff_location_id", "=", sample_id),
        ],
        columns=[
            "pickup_at",
            "dropoff_at",
            "pickup_location_id",
            "dropoff_location_id",
            "passenger_count",
            "trip_distance",
            "fare_amount",
        ],
    ).to_pandas()

    return df


# Function to transform a pandas dataframe
def transform_df(the_df: pd.DataFrame) -> pd.DataFrame:
    df = the_df.copy()
    # calculate trip_duration
    df["trip_duration"] = (df["dropoff_at"] - df["pickup_at"]).dt.seconds
    # filter trip_durations > 1 minute and less than 24 hours
    df = df[df["trip_duration"] > 60]
    df = df[df["trip_duration"] < 24 * 60 * 60]
    # keep only necessary columns
    df.drop(
        ["dropoff_at", "pickup_at", "pickup_location_id", "fare_amount"],
        axis=1,
        inplace=True,
    )
    df["dropoff_location_id"] = df["dropoff_location_id"].fillna(-1)
    return df

In [8]:
# %%time

# # Test reading data.
# import itertools
# my_list = itertools.product(s3_files, sample_locations)

# # [print(f[0], f[1]) for f in my_list]
# df_list = [read_data(f[0], f[1]) for f in my_list]
# df_raw = pd.concat(df_list, ignore_index=True)
# # Transform data.
# df = transform_df(df_raw)

# # Inspect the pandas dataframe.
# df.head()

## Define your Ray Tune Search Space and Search Algorithm <a class="anchor" id="define_search_space"></a>

In this notebook, we will use Ray Tune to run parallel training jobs per dropoff location.  The training jobs will be defined using a search space and simple grid search.  Depending on your need, fancier search spaces and search algorithms are possible with Tune. 

**First, define a search space of experiment trials to run.**  
> The typical use case for Tune search spaces are for hypterparameter tuning.  In our case, we are defining a Tune search space in a way to allow for training jobs to be conducted automatically.  Each training job will run on a different data partition (taxi dropoff location) and use a different model.

Common search algorithms include grid search, random search, and Bayesian optimization.  For more details, see [Working with Tune Search Spaces](https://docs.ray.io/en/master/tune/tutorials/tune-search-spaces.html#tune-search-space-tutorial).  Deciding the best combination of search space and search algorithm is part of the art of being a Data Scientist and depends on the data, algorithm, and problem being solved!  

**Next, define a search algorithm.**
>Ray Tune will use the search space and the specified search algorithm to generate multiple configurations, each of which will be evaluated in a separate Trial on a Ray Cluster. Ray Tune will take care of orchestrating those Trials automatically.  Specifically, Ray Tune will pass a config dictionary to each partition and make a Trainable function call.

**Below, we define our search space consists of:**
- 2 different Scikit-learn algorithms 
- Some or all NYC taxi drop-off locations. 

**Also below, we define our search algorithm is:**
- Grid search.

What this means is every algorithm will be applied to every NYC Taxi drop-off location.

In [9]:
# 1. Define a search space.
search_space = {
    "model": tune.grid_search(
        [LinearRegression(fit_intercept=True), DecisionTreeRegressor(max_depth=3)]
    ),
    "location": tune.grid_search(sample_locations),
}

## Define a Trainable (callable) function <a class="anchor" id="define_trainable"></a>

📈 Typically when you are running Data Science experiments, you want to be able to keep track of summary metrics for each trial, so you can decide at the end which trials were best.  That way, you can decide which model to deploy.

🇫 Next, we define a trainable function in order to train and evaluate a scikit-learn model on a data partition.  This function will be called in parallel by every Tune trial.  Inside this trainable function, we will:
- Add detailed metrics we want to report (each model's loss or error). 
- Checkpoint each model for easy deployment later.

📖 **The metrics defined inside the trainable function will appear in the Ray Tune experiment summary table.**
```{tip}
Ray Tune has two ways of defining a trainable, namely the [Function API](https://docs.ray.io/en/latest/tune/api_docs/trainable.html#trainable-docs) and the Class API. Both are valid ways of defining a trainable, but *the Function API is generally recommended*.
```

**In the cell below, we define a "Trainable" function called `train_model()`**.  
- The input is a config dictionary argument. 
- The output can be a simple dictionary of metrics which will be reported back to Tune.  
- We will [checkpoint](https://docs.ray.io/en/master/ray-air/key-concepts.html#checkpoints) save each model in addition to reporting each trial's metrics.
  > For checkpointing, we use `ray.air.checkpoint.Checkpoint`.  *Ray AIR includes integrations to popular ML libraries, including Scikit-learn*.  This makes it possible to use the convenient AIR API abstractions, without having to specify code details of the Scikit-learn library itself.
- Since we are using **grid search**, this means `train_model()` will be run *in parallel for every permutation* in the Tune search space!

In [10]:
from ray.air import Checkpoint

# 2. Define a custom train function
def train_model(config: dict):

    model = config["model"]
    location_id = config["location"]

    # Load data.
    df_list = [read_data(f, location_id) for f in s3_files]
    df_raw = pd.concat(df_list, ignore_index=True)
    df = transform_df(df_raw)

    # We need at least 10 rows to create a train / test split.
    if len(df) < 10:
        print_time(
            f"Data batch for LocID {location_id} is empty or smaller than 10 rows"
        )
        session.report(dict(error=None))
        return

    # Train/valid split.
    train_df, valid_df = train_test_split(df, test_size=0.2, shuffle=True)
    train_X = train_df[["passenger_count", "trip_distance"]]
    train_y = train_df.trip_duration
    valid_X = valid_df[["passenger_count", "trip_distance"]]
    valid_y = valid_df.trip_duration

    # Train model.
    model = model.fit(train_X, train_y)
    pred_y = model.predict(valid_X)

    # Evaluate.
    error = sklearn.metrics.mean_absolute_error(valid_y, pred_y)

    # Define a model checkpoint using AIR API.
    # https://docs.ray.io/en/latest/tune/tutorials/tune-checkpoints.html
    checkpoint = Checkpoint.from_dict({"model": model, "location_id": location_id})

    # Save checkpoint and report back metrics, using ray.air.session.report()
    # The metrics you specify here will appear in Tune summary table.
    # They will also be recorded in Tune results under `metrics`.
    metrics = dict(error=error)
    session.report(metrics, checkpoint=checkpoint)

## Run batch training on Ray Tune <a class="anchor" id="run_tune_search"></a>

**In the cell below, we configure the resources allocated per trial.** 

Tune uses this resources allocation to control the parallelism. For example, if each trial was configured to use 4 CPUs, and the cluster had only 32 CPUs, then Tune will limit the number of concurrent trials to 8 to avoid overloading the cluster. For more information, see [A Guide To Parallelism and Resources](https://docs.ray.io/en/master/tune/tutorials/tune-resources.html#tune-parallelism).

In [11]:
# 3. Customize resources per trial, here we set 1 CPU each.
train_model = tune.with_resources(train_model, {"cpu": 1})

<br>

**Now we are ready to kick off a Ray Tune experiment!**  

Recall what we are doing, high level, is training several different models per dropoff location.  We are using Ray Tune so we can run all these trials in parallel.  At the end, we will inspect the results of the experiment and deploy only the best model per dropoff location.

**In the cell below, we use AIR configs and run the experiment using `tuner.fit()`.** 

Tune will report on experiment status, and after the experiment finishes, you can inspect the results. 

In the AIR config below, we have specified a local directory `my_Tune_logs` for logging instead of the default `~/ray_results` directory. Giving your logs a project name makes them easier to find.  Also giving a relative path, means you can see your logs inside the Jupyter browser.  Learn more about logging Tune results at [How to configure logging in Tune](https://docs.ray.io/en/master/tune/tutorials/tune-output.html#tune-logging).

Tune can [retry failed experiments automatically](https://docs.ray.io/en/master/tune/tutorials/tune-stopping.html#tune-stopping-guide), as well as entire experiments.  This is necessary in case a node on your remote cluster fails (when running on a cloud such as AWS or GCP).

💡 Right-click on the cell below and choose "Enable Scrolling for Outputs"! This will make it easier to view, since model training output can be very long!

**Below was tested for 518 models, using 18 NYC Taxi S3 files dating from 2018/01 to 2019/06 (split into partitions approx 1GiB each), simultaneously trained on a 23-node AWS cluster of [m5.4xlarges](https://aws.amazon.com/ec2/instance-types/m5/).  Total data reading and train time was 37 minutes.**

In [21]:
# Define a tuner object using Ray AIR Tuner API
tuner = tune.Tuner(
    train_model,
    param_space=search_space,
    run_config=air.RunConfig(
        # redirect logs to relative path instead of default ~/ray_results/
        local_dir="my_Tune_logs",
        name="batch_tuning",
        # Set Ray Tune verbosity.  Print summary table only with levels 2 or 3.
        verbose=2,
    ),
)

# 4. Run the experiment with Ray Tune
start = time.time()
results = tuner.fit()
total_time_taken = time.time() - start

# Print some training stats
print(f"Total number of models: {len(results)}")
print(f"TOTAL TIME TAKEN: {total_time_taken:.2f} seconds")
best_result = results.get_best_result(metric="error", mode="min").config
print(f"Best result: {best_result}")

Trial name,error,should_checkpoint
train_model_cabb8_00000,447.234,True
train_model_cabb8_00001,493.912,True
train_model_cabb8_00002,461.717,True
train_model_cabb8_00003,832.879,True
train_model_cabb8_00004,447.743,True
train_model_cabb8_00005,446.824,True
train_model_cabb8_00006,424.082,True
train_model_cabb8_00007,496.27,True
train_model_cabb8_00008,502.505,True
train_model_cabb8_00009,473.44,True


Total number of models: 518
TOTAL TIME TAKEN: 2223.62 seconds
Best result: {'model': DecisionTreeRegressor(max_depth=3), 'location': 199}


<br>

**After the Tune experiment has run, select the best model per dropoff location.**

We can assemble the Tune results ([ResultGrid object](https://docs.ray.io/en/master/tune/examples/tune_analyze_results.html)) into a pandas dataframe, then sort by minimum error, to select the best model per dropoff location.

In [22]:
# get a list of training loss errors
errors = []
[errors.append(i.metrics.get("error", 10000.0)) for i in results]

# get a list of checkpoints
checkpoints = []
[checkpoints.append(i.checkpoint) for i in results]

# get a list of locations
locations = []
[locations.append(i.config["location"]) for i in results]

# get a list of models
models = []
[models.append(i.config["model"]) for i in results]

# Assemble a pandas dataframe from Tune results
results_df = pd.DataFrame(
    zip(locations, models, errors, checkpoints),
    columns=["location_id", "model", "error", "checkpoint"],
)
print(results_df.dtypes)
results_df.head()

location_id      int64
model           object
error          float64
checkpoint      object
dtype: object


,location_id,model,error,checkpoint
0,24,LinearRegression(),447.234058,Checkpoint(local_path=/home/ray/christy-air/my...
1,140,LinearRegression(),493.912464,Checkpoint(local_path=/home/ray/christy-air/my...
2,141,LinearRegression(),461.717301,Checkpoint(local_path=/home/ray/christy-air/my...
3,257,LinearRegression(),832.879303,Checkpoint(local_path=/home/ray/christy-air/my...
4,239,LinearRegression(),447.742891,Checkpoint(local_path=/home/ray/christy-air/my...


In [23]:
# Keep only 1 model per location_id with minimum error
final_df = results_df.dropna(subset=["error"])
final_df = final_df.loc[results_df.groupby("location_id")["error"].idxmin()].copy()
final_df.sort_values(by=["error"], inplace=True)
final_df.set_index("location_id", inplace=True, drop=True)
print(final_df.dtypes)
final_df

model          object
error         float64
checkpoint     object
dtype: object


,model,error,checkpoint
location_id,,,
199,DecisionTreeRegressor(max_depth=3),147.133333,Checkpoint(local_path=/home/ray/christy-air/my...
105,DecisionTreeRegressor(max_depth=3),247.749018,Checkpoint(local_path=/home/ray/christy-air/my...
236,DecisionTreeRegressor(max_depth=3),369.222274,Checkpoint(local_path=/home/ray/christy-air/my...
238,DecisionTreeRegressor(max_depth=3),384.026867,Checkpoint(local_path=/home/ray/christy-air/my...
207,LinearRegression(),396.891111,Checkpoint(local_path=/home/ray/christy-air/my...
...,...,...,...
217,DecisionTreeRegressor(max_depth=3),1326.447774,Checkpoint(local_path=/home/ray/christy-air/my...
5,LinearRegression(),1417.664071,Checkpoint(local_path=/home/ray/christy-air/my...
6,DecisionTreeRegressor(max_depth=3),1543.497932,Checkpoint(local_path=/home/ray/christy-air/my...


## Load a model from checkpoint and perform inference  <a class="anchor" id="load_checkpoint"></a>

```{tip}
[Ray AIR Predictors](https://docs.ray.io/en/latest/ray-air/predictors.html) make batch inference easy since they have internal logic to parallelize the inference.
```
  
In this notebook, we will restore a single scikit-learn model directly from checkpoint, and demonstrate it can be used for inference.

Below, we can easily obtain AIR Checkpoint objects from the Tune results. 

In [24]:
# Choose a dropoff location
location_id = 90
location_id

90

In [25]:
# Get a checkpoint directly from the pandas dataframe of Tune results
checkpoint = final_df.checkpoint[location_id]
print(type(checkpoint))

# Restore a model from checkpoint
model = checkpoint.to_dict()["model"]
print(type(model))

<class 'ray.air.checkpoint.Checkpoint'>
<class 'sklearn.tree._classes.DecisionTreeRegressor'>


In [26]:
# Create some test data
df_list = [read_data(f, location_id) for f in s3_files[:1]]
df_raw = pd.concat(df_list, ignore_index=True)
df = transform_df(df_raw)

# Train/test split.
_, test_df = train_test_split(df, test_size=0.2, shuffle=True)
test_X = test_df[["passenger_count", "trip_distance"]]
test_y = np.array(test_df.trip_duration)  # actual values

In [27]:
# Perform inference using restored model from checkpoint
pred_y = model.predict(test_X)

# Zip together predictions and actuals to visualize
pd.DataFrame(zip(pred_y, test_y), columns=["pred_y", "trip_duration"])[0:10]

,pred_y,trip_duration
0,880.224200,753
1,880.224200,699
2,364.604675,408
3,600.984340,259
4,600.984340,451
5,2221.970460,1144
6,1197.244840,483
7,880.224200,564
8,880.224200,546
9,600.984340,466


**Compare validation and test error.**

During model training we reported error on "validation" data (random sample).  Below, we will report error on a pretend "test" data set (a different random sample).

Do a quick validation that both errors are reasonably close together.

In [28]:
# Evaluate restored model on test data.
error = sklearn.metrics.mean_absolute_error(test_y, pred_y)
print(f"Test error: {error}")

Test error: 404.4554806162227


In [29]:
# Compare test error with training validation error
print(f"Validation error: {final_df.error[location_id]}")

# Validation and test errors should be reasonably close together.

Validation error: 452.53533680221614
